## Flood Health Vulnerability (FHV) Assessment 2
This notebooks applies FHV assessment for the 2020 flood event.

In [2]:
import os
import sys
from functools import reduce
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
import geopandas as gpd
import rasterio
import fhv
# Mapping
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patheffects as pe
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set_style("white", {'axes.linewidth': 1, 'grid.color': 'black'})
lims = [88.00, 92.7, 20.57, 26.64]    # [xmin, xmax, ymin, ymax]

# ADMINISTRATIVE SHAPEFILE
# ------------------------------------------------- #
# Upazila
shape3 = gpd.read_file('./data/admin_boundary/bgd_admbnda_adm3_bbs_20180410.shp')
# Convert ADM3_PCODE of Mymensingh (45) division (total 378 unions) (45 -> 30)
f45t30 = '30' + shape3.loc[shape3['ADM1_PCODE'] == '45', 'ADM3_PCODE'].str[2:]
shape3.loc[shape3['ADM1_PCODE'] == '45', 'ADM3_PCODE'] = f45t30.values
shape3['ADM3_PCODE'] = shape3['ADM3_PCODE'].astype(int)
f45t30 = '30' + shape3.loc[shape3['ADM1_PCODE'] == '45', 'ADM2_PCODE'].str[2:]
shape3.loc[shape3['ADM1_PCODE'] == '45', 'ADM2_PCODE'] = f45t30.values
shape3['ADM2_PCODE'] = shape3['ADM2_PCODE'].astype(int)
shape3 = shape3.drop(shape3.columns[[0,1,4,5,6,9,10,11,12,13,14,15]], axis=1)
ADM2 = shape3[['ADM2_EN','ADM2_PCODE']].copy().drop_duplicates()
ADM2['ADM2_PCODE'] = ADM2['ADM2_PCODE'].astype(int)
# District
shape2 = gpd.read_file('./data/admin_boundary/bgd_admbnda_adm2_bbs_20180410.shp')
# Convert ADM3_PCODE of Mymensingh (45) division (total 378 unions) (45 -> 30)
shape2 = shape2.drop(shape2.columns[[0,1,4,5,6,9,10,11,12,13]], axis=1)
f45t30 = '30' + shape2.loc[shape2['ADM1_PCODE'] == '45', 'ADM2_PCODE'].str[2:]
shape2.loc[shape2['ADM1_PCODE'] == '45', 'ADM2_PCODE'] = f45t30.values
shape2['ADM2_PCODE'] = shape2['ADM2_PCODE'].astype(int)
# Division
shape1 = gpd.read_file('./data/admin_boundary/bgd_admbnda_adm1_bbs_20180410.shp')
# ------------------------------------------------- #

# District-level Population
df = pd.read_excel('./data/disaster_records/damagedata_DDM.xlsx', 
                   sheet_name='total_population',skiprows=0,skipfooter=0).fillna(0)
popu2017 = df[['ADM2_PCODE', 'Population']]

# Load Vulnerability
vuln = pd.read_hdf('./data/vuln_calculated_adm3.hdf')

## Generate 2020 FFWC Flood Forecast

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling
def ReprojectToReference(in_path, ref_path, out_path, out_dtype, scale=1, out_nodata=None):
    """Reproject a raster to reference raster
    """
    with rasterio.open(ref_path) as ref:
        profile = ref.profile.copy()
        profile.update(
            dtype=out_dtype,
            nodata=out_nodata,
            compress='lzw')
        with rasterio.open(in_path) as src:
            with rasterio.open(out_path, 'w', **profile) as dst:
                for i in range(1, src.count + 1):
                    source_data = src.read(i)
                    source_data[source_data>=0] = source_data[source_data>=0]*scale
                    source_data[source_data<0] = 0
                    reproject(
                        source=source_data,
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=dst.transform,
                        dst_crs=dst.crs,
                        resampling=Resampling.average)
                print("%s is saved." % out_path)


# FFWC Flood Forecast (Inundation) Map issued at July-12, 2020
ref_path = './data/bgd_ppp_2017_30m_decuple.tif'
for i in range(1,7):
    in_path = './data/flood2020/Raster_12-07-2020/extract_floo%d'%i
    out_path = './data/flood2020/Raster_12-07-2020/extract_floo%d_decuple.tif'%i
    ReprojectToReference(in_path,ref_path,out_path,rasterio.uint32,scale=10,out_nodata=65536)

./data/flood2020/Raster_12-07-2020/extract_floo1_decuple.tif is saved.
./data/flood2020/Raster_12-07-2020/extract_floo2_decuple.tif is saved.


## Join 2020 FFWC Flood Forecast and Inundation to Population data

In [ ]:
# Union ID list (5160)
uidList = pd.read_hdf('./data/uidlist.hdf')
# Flatten cell indices for Union ID
uidx = np.load('./data/uidx.npz', allow_pickle=True)['uidx']
# Shapefiles of administrative units
shp_district = gpd.read_file('./data/adm_district.shp')
shp_upazila = gpd.read_file('./data/adm_upazila.shp')
shp_union = gpd.read_file('./data/adm_union.shp')

# Load data
popu = rasterio.open('./data/bgd_ppp_2017_30m_decuple.tif').read(1).flatten()    # Population Per Pixel (Decuple value)
fcst = rasterio.open('./data/flood2020/Raster_12-07-2020/extract_floo6_decuple.tif').read(1).flatten()  # Flood depth (decuple; unit is 10 cm)

In [ ]:
# Affected population by Flood Inundation and Flood Forecast
# (b) Flood Forecast        # Flood depth (decuple; unit is 10 cm)
# *Affected population increase linearly with flood depth by 3m
popuFcst = popu.copy(); thsd = 35                          
popuFcst[fcst <= thsd] =  popuFcst[fcst <= thsd]*(fcst[fcst <= thsd])**2/thsd**2
popuFcst[fcst < 10] = 0
popuFcst[fcst > 60000] = 0

# Counting population
totlPopu = np.zeros(uidList.shape)
afftPopuFcst = np.zeros_like(totlPopu)
# totlCell = [len(udx) for udx in uidx]
# afftCellFcst = np.zeros_like(totlPopu)
for i, uid in uidList.iteritems():
    totlPopu[i] = popu[uidx[i]].sum()
    afftPopuFcst[i] = popuFcst[uidx[i]].sum()

# Rescale from decuple to normal and Add one to avoid numeric errors
totlPopu = (totlPopu+1)/10
afftPopuFcst /= 10

# Generate a Union dataframe
impact4 = shp_union.drop('geometry', axis=1)
impact4['POP'], impact4['POPFCST'] = totlPopu, afftPopuFcst
impact4['POPFCSTR'] = afftPopuFcst/totlPopu
# Aggregate to Upazila level
varlist = ['POP','POPFCST']
impact3 = impact4.groupby(['ADM3_PCODE', 'ADM3_EN'])[varlist].sum()
impact3['POPFCSTR'] = impact3['POPFCST']/impact3['POP']
impact3['ADM2_PCODE'], impact3['ADM2_EN'] = shp_upazila['ADM2_PCODE'].values, shp_upazila['ADM2_EN'].values

# Print
print('POPFCST: {:,}'.format(impact3['POPFCST'].sum().astype(int)))

In [ ]:
temp = impact4.copy().reset_index()
temp = temp[[a.startswith('2015') for a in temp['ADM4_PCODE'].values]]
# temp['POPFCST'].values

## District-level averaged domain vulnerability, FHV, and affected population

In [ ]:
# Affected popoulation in Upazila 
impact = impact3.reset_index().drop(['ADM2_PCODE','ADM2_EN','ADM3_EN'], axis=1)
impact['ADM3_PCODE'] = impact['ADM3_PCODE'].astype(int)
# Convert ADM3_PCODE of Mymensingh (45) division (total 378 unions) (45 -> 30)
targ = (impact['ADM3_PCODE']/10000).astype(int) == 45
f45t30 = 300000 + impact.loc[targ, 'ADM3_PCODE'] % 10000
impact.loc[targ, 'ADM3_PCODE'] = f45t30.values
impact['ADM2_PCODE'] = (impact['ADM3_PCODE'] / 100).astype(int)
impact = impact.set_index('ADM3_PCODE')

# Merge vulnerability
merge3 = pd.merge(impact, vuln, how='outer', left_index=True, right_index=True)

# IMPACT ASSESSMENT -------------------------- #
# Multiplication (will be divided by district population)
# - Total population
merge3['EQUAL'] = merge3['POP']*merge3['equal']
# - Predicted affected population (forecast and inundation)
merge3['EQUALF'] = merge3['POPFCST']*merge3['equal']
# District-level aggregation
cols = ['POPFCST','EQUAL','EQUALF']
merge2 = merge3.groupby('ADM2_PCODE')[cols].sum().reset_index()
# - Import population
merge2 = pd.merge(popu2017, merge2, how='outer',on='ADM2_PCODE')
merge2 = merge2.rename(columns={'Population':'POP'}).set_index('ADM2_PCODE')
# - Calculation
merge2['EQUAL'] = merge2['EQUAL']/merge2['POP']
merge2['EQUALF'] = merge2['EQUALF']/(merge2['POPFCST']+1)
# -------------------------------------------- #

# GeoDataFrame
merge3 = merge3.drop('ADM2_PCODE', axis=1)
mapdata3 = shape3.merge(merge3, how='outer', on='ADM3_PCODE')
mapdata2 = shape2.merge(merge2, how='outer', on='ADM2_PCODE')

In [ ]:
result = pd.merge(merge2, shape2[['ADM2_PCODE', 'ADM2_EN']], on='ADM2_PCODE')
result = result[['ADM2_PCODE','ADM2_EN','EQUAL','EQUALF']]
result = result.rename(columns={'EQUAL': 'Original', 'EQUALF': 'Weighted'})
result.to_excel('./data/fhv_jul2020_flood.xlsx')

### District-level Average FHV of Affected Population

In [ ]:
# Colormap
bounds = list(np.arange(0.2,0.8,0.1))
boundaries = [0]+bounds+[1]
cmap, norm, vmin, vmax, ticks, boundaries = fhv.cbarpam(bounds, 'RdBu_r', labloc='on',boundaries=boundaries, extension='both')

# Text
text = "Social-Health Vulnerability (SHV) is determined by weighting 26 socio-economic, health, and\ncoping capacity indicators equally.\
 The left map is an original SHV, and the right map is an\naffected population-weighted SHV based on FFWC's July 2020 flood forecast.\
 \n\nContacts to Donghoon Lee (donghoon.lee@wisc.edu)"


# Plotting
sns.set_style("white", {'axes.linewidth': 1, 'grid.color': 'black'})
# fignumb = ['(a)', '(b)']
figlabel = 'Social-Health\nVulnerability'
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(10,5), facecolor='w')
for (i, el) in enumerate(['EQUAL','EQUALF']):
    ax = axes.flatten('F')[i]
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.axis(lims)
    mapdata2.plot(ax=ax, column=el,cmap=cmap, norm=norm, vmin=vmin, vmax=vmax, 
                     linewidth=0.3, edgecolor='grey')
#     ax.annotate(fignumb[i], xy=(-0.01, 0.96), xycoords='axes fraction',
#                 horizontalalignment='center', verticalalignment='center',
#                 fontname='arial',fontsize=20)
    # Borderline of ADM1
    shape1.boundary.plot(ax=ax, linewidth=1.2, edgecolor='#525252')
plt.tight_layout(pad=-5)
ax.annotate(text, xy=(-1., -0.1), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='center',
                fontname='arial',fontsize=18)


# Colorbar
cax = inset_axes(ax, width="5%", height="40%",
                 loc='lower left',
                 bbox_to_anchor=(1.05, 0.27, 0.6, 1.2),
                 bbox_transform=ax.transAxes,borderpad=0)
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm,
                                 boundaries=boundaries,
                                 extend='both',
                                 extendfrac=0.08,
                                 ticks = ticks,
                                 spacing='uniform',
                                 orientation='vertical')
cbar.ax.set_yticklabels(['%.1f'%lb for lb in np.array(bounds)], 
                        fontname='arial', fontsize=14)
cbar.ax.annotate(figlabel, xy=(3.8, 1.15), xycoords='axes fraction',
                horizontalalignment='right', verticalalignment='center',
                fontname='arial',fontsize=16)    
plt.show()

# Save a figure
if True:
    fn_save = './figures/fhv_district_2020_ffwc.pdf'
    fig.savefig(fn_save, bbox_inches='tight'); print('%s is saved.' % fn_save)
    fn_save = './figures/fhv_district_2020_ffwc.png'
    fig.savefig(fn_save, bbox_inches='tight'); print('%s is saved.' % fn_save)

In [ ]:
result